In [3]:
# MENGIMPOR SEMUA LIBRARY YANG DIPERLUKAN UNTUK BAB 4
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone # Diperlukan untuk early stopping

# Mengatur default plotting
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Untuk plot di dalam notebook (jika Anda menggunakan Jupyter)
# %matplotlib inline

print("--- BAB 4: MELATIH MODEL ---")
print("Semua library telah diimpor.")

# 1. REGRESI LINEAR
# Mari kita fokus pada cara melatih model linear.
# Modelnya: y_hat = theta_0 + theta_1*x_1 + ... + theta_n*x_n
# Cost Function (Fungsi Biaya): Mean Squared Error (MSE)
print("\n[1. Regresi Linear]")

# Membuat data linear palsu untuk pengujian
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1) # y = 4 + 3x + noise

# 1a. Metode 1: The Normal Equation (Persamaan Normal)
print("  Melatih dengan Persamaan Normal...")
# Teori: Persamaan Normal adalah solusi analitik (rumus pasti)
# untuk menemukan nilai theta yang meminimalkan cost function.
# Rumus: theta_best = (X_transpose * X)^-1 * X_transpose * y
# Kita perlu menambahkan x0 = 1 (bias term) ke setiap instance.
X_b = np.c_[np.ones((100, 1)), X]  # np.c_ adalah concatenate
theta_best_normal = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

print(f"  Theta terbaik (dari Persamaan Normal): \n{theta_best_normal}")
# Hasilnya harusnya dekat dengan [4] dan [3]

# Mari kita prediksi menggunakan theta ini
# X_new = np.array([[0], [2]])
# X_new_b = np.c_[np.ones((2, 1)), X_new] # Tambahkan x0 = 1
# y_predict = X_new_b.dot(theta_best_normal)
# print(f"  Prediksi: \n{y_predict}")

# Plot hasil Persamaan Normal (di-comment)
# plt.plot(X_new, y_predict, "r-")
# plt.plot(X, y, "b.")
# plt.axis([0, 2, 0, 15])
# plt.title("Regresi Linear via Persamaan Normal")
# plt.show()

# 1b. Menggunakan Scikit-Learn LinearRegression
# Scikit-Learn melakukan hal yang sama (menggunakan SVD, bukan inversi matriks)
lin_reg = LinearRegression()
lin_reg.fit(X, y)
print(f"  Theta terbaik (dari Scikit-Learn): \n  Intercept: {lin_reg.intercept_}, Coef: {lin_reg.coef_}")

# 1c. Metode 2: Gradient Descent (GD)
print("  Melatih dengan Gradient Descent...")
# Teori: GD adalah algoritma optimasi iteratif.
# Ia menyesuaikan parameter (theta) sedikit demi sedikit untuk
# menemukan nilai minimum dari Cost Function (MSE).
# Hyperparameter kuncinya adalah 'learning_rate' (eta).

# PENTING: GD SANGAT SENSITIF PADA SKALA FITUR.
# Kita harus melakukan feature scaling (misal StandardScaler)
# sebelum menggunakan GD. Namun, karena data X kita skalanya sudah
# kecil (0-2), kita bisa melewatinya untuk contoh sederhana ini.

# Varian 1: Batch Gradient Descent (BGD)
# BGD menggunakan *seluruh* dataset latih pada setiap langkah.
eta = 0.1  # Learning rate
n_iterations = 1000
m = 100 # Jumlah instance
theta_bgd = np.random.randn(2, 1)  # Inisialisasi acak

for iteration in range(n_iterations):
    # Hitung gradien (turunan parsial dari MSE)
    gradients = 2/m * X_b.T.dot(X_b.dot(theta_bgd) - y)
    # Update theta (melangkah "menuruni" bukit)
    theta_bgd = theta_bgd - eta * gradients

print(f"  Theta terbaik (dari Batch GD): \n{theta_bgd}")

# Varian 2: Stochastic Gradient Descent (SGD)
# SGD mengambil *satu* instance acak pada setiap langkah.
# Jauh lebih cepat, tapi kurang stabil (stochastic/acak).
# Perlu "learning schedule" untuk mengurangi 'eta' seiring waktu.
print("  Melatih dengan Stochastic GD (SGD)...")
n_epochs = 50
t0, t1 = 5, 50  # Hyperparameter untuk learning schedule

def learning_schedule(t):
    return t0 / (t + t1)

theta_sgd = np.random.randn(2, 1)  # Inisialisasi acak

for epoch in range(n_epochs): # Epoch = satu putaran penuh data
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]

        gradients = 2 * xi.T.dot(xi.dot(theta_sgd) - yi)
        eta = learning_schedule(epoch * m + i)
        theta_sgd = theta_sgd - eta * gradients

print(f"  Theta terbaik (dari SGD): \n{theta_sgd}")

# Varian 3: Mini-batch Gradient Descent (MGD)
# Kompromi: menghitung gradien pada 'mini-batch' (sekelompok kecil data).
# Ini adalah yang paling umum digunakan dalam Deep Learning.

# Scikit-Learn SGDRegressor mengimplementasikan SGD/MGD
# (Secara default MGD karena batch_size default tidak 1)
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=42)
# y.ravel() mengubah y dari [100, 1] menjadi [100,]
sgd_reg.fit(X, y.ravel())
print(f"  Theta terbaik (dari SGDRegressor SK-Learn): \n  Intercept: {sgd_reg.intercept_}, Coef: {sgd_reg.coef_}")


# 2. REGRESI POLINOMIAL
print("\n[2. Regresi Polinomial]")
# Teori: Kita bisa menggunakan model linear untuk data non-linear
# dengan cara menambahkan fitur baru yang merupakan pangkat dari fitur asli.

# Membuat data non-linear (kuadratik)
m = 100
X_poly_data = 6 * np.random.rand(m, 1) - 3
y_poly_data = 0.5 * X_poly_data**2 + X_poly_data + 2 + np.random.randn(m, 1)

# Plot data non-linear (di-comment)
# plt.plot(X_poly_data, y_poly_data, "b.")
# plt.xlabel("$x_1$")
# plt.ylabel("$y$")
# plt.title("Data Non-Linear")
# plt.axis([-3, 3, 0, 10])
# plt.show()

# Gunakan Scikit-Learn PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly_transformed = poly_features.fit_transform(X_poly_data)

# X_poly_transformed sekarang punya 2 fitur: x1 dan x1^2
# print(f"  Fitur asli: {X_poly_data[0]}")
# print(f"  Fitur baru (polinomial): {X_poly_transformed[0]}")

# Latih LinearRegression pada data yang sudah ditransformasi
lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_poly_transformed, y_poly_data)
print(f"  Theta (polinomial): Intercept: {lin_reg_poly.intercept_}, Coef: {lin_reg_poly.coef_}")
# Hasilnya harusnya dekat dengan [2] (intercept), [1] (coef x1), [0.5] (coef x1^2)


# 3. KURVA BELAJAR (LEARNING CURVES)
print("\n[3. Kurva Belajar (Learning Curves)]")
# Teori: Kurva Belajar memplot error latih & error validasi
# sebagai fungsi dari ukuran data latih.
# Ini adalah cara utama untuk mendiagnosis Overfitting atau Underfitting.

def plot_learning_curves(model, X, y):
    # Membagi data untuk validasi
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
    train_errors, val_errors = [], []

    for m in range(1, len(X_train)): # Iterasi dari 1 instance s/d semua
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="Train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="Validation")
    plt.legend(loc="upper right", fontsize=14)
    plt.xlabel("Training set size", fontsize=14)
    plt.ylabel("RMSE", fontsize=14)
    plt.axis([0, 80, 0, 3])

# Contoh 1: Underfitting (Model terlalu sederhana)
# lin_reg_curve = LinearRegression()
# plot_learning_curves(lin_reg_curve, X_poly_data, y_poly_data)
# plt.title("Kurva Belajar (Underfitting)")
# plt.show()
# Hasil: Kedua error bertemu di plato pada level error yang TINGGI.

# Contoh 2: Overfitting (Model terlalu kompleks)
# polynomial_regression = Pipeline([
#     ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
#     ("lin_reg", LinearRegression()),
# ])
# plot_learning_curves(polynomial_regression, X_poly_data, y_poly_data)
# plt.title("Kurva Belajar (Overfitting)")
# plt.show()
# Hasil: Ada JARAK (gap) besar antara error latih (rendah) dan error validasi (tinggi).

# 4. MODEL LINEAR YANG DIREGULARISASI
print("\n[4. Model Linear yang Diregularisasi]")
# Teori: Regularisasi adalah cara untuk membatasi (constrain)
# model untuk mencegah overfitting.

# 4a. Ridge Regression (Penalti L2)
# Menambahkan penalty L2 (setengah kuadrat dari norma L2 bobot) ke cost function.
# Ini memaksa bobot (weights) tetap kecil.
from sklearn.linear_model import Ridge
# alpha = kekuatan regularisasi. alpha=0 sama dengan Linear Regression.
ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
ridge_reg.fit(X, y)
# print(ridge_reg.predict([[1.5]]))

# 4b. Lasso Regression (Penalti L1)
# Menambahkan penalty L1 (norma L1 dari bobot) ke cost function.
# Efek utamanya: cenderung menghilangkan bobot fitur yang tidak penting (menjadi 0).
# Bagus untuk feature selection otomatis.
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
# print(lasso_reg.predict([[1.5]]))

# 4c. Elastic Net
# Gabungan dari Ridge dan Lasso.
# Dikontrol oleh 'l1_ratio' (r). Jika r=1, ini Lasso. Jika r=0, ini Ridge.
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_net.fit(X, y)
# print(elastic_net.predict([[1.5]]))

# 4d. Early Stopping (Penghentian Dini)
# Teori: Bentuk regularisasi yang sangat berbeda.
# Hentikan pelatihan segera setelah performa di validation set
# berhenti membaik (mulai naik).
print("  Implementasi Early Stopping (konseptual)...")
# (Kode implementasi manual Early Stopping akan panjang,
#  jadi kita hanya tunjukkan konsep dasarnya)
# (Kita tiru data dari buku)
np.random.seed(42)
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)
X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size=0.5, random_state=10)

# Siapkan data
poly_scaler = Pipeline([
    ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
    ("std_scaler", StandardScaler())
])
X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.inf, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

minimum_val_error = float("inf")
best_epoch = None
best_model = None

# for epoch in range(1000):
#     sgd_reg.fit(X_train_poly_scaled, y_train)  # warm_start=True melanjutkan latihan
#     y_val_predict = sgd_reg.predict(X_val_poly_scaled)
#     val_error = mean_squared_error(y_val, y_val_predict)
#     if val_error < minimum_val_error:
#         minimum_val_error = val_error
#         best_epoch = epoch
#         best_model = clone(sgd_reg) # Simpan model terbaik

# print(f"  Epoch terbaik: {best_epoch}")
# print(f"  Error validasi minimum: {minimum_val_error}")


# 5. LOGISTIC & SOFTMAX REGRESSION
print("\n[5. Regresi Logistik dan Softmax]")
# Teori: Logistic Regression adalah untuk klasifikasi biner.
# Ia menghitung probabilitas menggunakan fungsi sigmoid (logistik).

# Menggunakan dataset Iris
iris = datasets.load_iris()
X_iris = iris["data"][:, (2, 3)]  # petal length, petal width
y_iris = iris["target"]

# 5a. Logistic Regression (Binary)
# Kita buat target biner: Apakah Iris virginica (kelas 2)?
y_iris_virginica = (y_iris == 2).astype(np.int64)

log_reg = LogisticRegression(solver="lbfgs", random_state=42)
log_reg.fit(X_iris, y_iris_virginica)

# X_new_iris = np.array([[1.7, 0.5], [5, 2]])
# print(f"  Prediksi (biner): {log_reg.predict(X_new_iris)}")
# print(f"  Probabilitas (biner): \n{log_reg.predict_proba(X_new_iris)}")


# 5b. Softmax Regression (Multiclass)
# Teori: Generalisasi Logistic Regression untuk banyak kelas.
# Menggunakan fungsi softmax untuk memastikan probabilitas semua kelas berjumlah 1.
# Cukup atur multi_class="multinomial"
softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X_iris, y_iris) # Latih pada semua 3 kelas

print(f"  Prediksi Softmax (multikelas) untuk [5, 2]: {softmax_reg.predict([[5, 2]])}")
print(f"  Probabilitas Softmax (multikelas) untuk [5, 2]: \n{softmax_reg.predict_proba([[5, 2]])}")

print("\n--- Selesai Bab 4 ---")

--- BAB 4: MELATIH MODEL ---
Semua library telah diimpor.

[1. Regresi Linear]
  Melatih dengan Persamaan Normal...
  Theta terbaik (dari Persamaan Normal): 
[[3.86501051]
 [3.13916179]]
  Theta terbaik (dari Scikit-Learn): 
  Intercept: [3.86501051], Coef: [[3.13916179]]
  Melatih dengan Gradient Descent...
  Theta terbaik (dari Batch GD): 
[[3.86501051]
 [3.13916179]]
  Melatih dengan Stochastic GD (SGD)...
  Theta terbaik (dari SGD): 
[[3.91897165]
 [3.13680279]]
  Theta terbaik (dari SGDRegressor SK-Learn): 
  Intercept: [3.86256592], Coef: [3.15101583]

[2. Regresi Polinomial]
  Theta (polinomial): Intercept: [1.69877159], Coef: [[1.0346329  0.54852217]]

[3. Kurva Belajar (Learning Curves)]

[4. Model Linear yang Diregularisasi]
  Implementasi Early Stopping (konseptual)...

[5. Regresi Logistik dan Softmax]
  Prediksi Softmax (multikelas) untuk [5, 2]: [2]
  Probabilitas Softmax (multikelas) untuk [5, 2]: 
[[6.21626374e-07 5.73689802e-02 9.42630398e-01]]

--- Selesai Bab 4 ---


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
